In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-11-09"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
19327,2024-11-09,Brasil Nbb,15:00,Brasília,Unifacisa,1.87,1.79,165.5,1.83,1.83,-1.5,2.05,1.65,WWz44LNr,0.534759,0.558659,0.546448,0.546448,0.093419,0.6,0.8,NaN,NaN,181.982,97.981643,0.538414,141.242,71.478120,0.506068,236.232,213.014,100.74,154.77,0.0,0.0,0.0,0.0,0.030912,0.000000,0.152888,-0.68,-0.136,-6.397059,0.431350,0.4,-0.031350,-3.34,-0.668,-1.182635,0.495284,0.3,-0.195284
19328,2024-11-09,Brasil Nbb,16:00,R10 Score Vasco da Gama,União Corinthians,1.61,2.12,146.5,1.83,1.83,-7.5,1.98,1.70,EFSK0wN7,0.621118,0.471698,0.546448,0.546448,0.092816,0.6,0.6,NaN,NaN,109.200,39.147222,0.358491,215.402,71.912957,0.333855,122.984,263.700,89.60,183.34,0.0,0.0,0.0,0.0,0.193364,0.000000,0.107603,-0.89,-0.178,-3.426966,0.650710,0.7,0.049290,2.94,0.588,1.904762,0.298883,0.4,0.101117
19329,2024-11-09,Brasil Nbb,17:00,Corinthians Paulista,Franca,2.48,1.45,161.5,1.83,1.83,3.5,2.04,1.66,69F6OtFE,0.403226,0.689655,0.546448,0.546448,0.092881,0.6,0.2,NaN,NaN,214.532,156.208986,0.728138,106.528,38.717625,0.363450,133.208,131.626,127.71,93.10,0.0,0.0,0.0,0.0,0.370646,0.000000,0.145244,-5.00,-1.000,-1.480000,0.618231,0.2,-0.418231,-1.35,-0.270,-1.666667,0.640553,0.7,0.059447
19330,2024-11-09,Brasil Nbb,21:00,Botafogo,Caxias do Sul,1.56,2.21,150.5,1.86,1.80,-4.5,1.99,1.70,ImZB2ahe,0.641026,0.452489,0.537634,0.555556,0.093514,0.6,0.2,NaN,NaN,235.092,115.169133,0.489890,339.608,163.534186,0.481538,311.958,366.848,128.64,352.87,0.0,0.0,0.0,0.0,0.243830,0.023184,0.111144,-2.82,-0.564,-0.992908,0.420654,0.2,-0.220654,-0.49,-0.098,-12.346939,0.282684,0.2,-0.082684
19331,2024-11-09,Espanha Acb,14:00,Manresa,Granada,1.17,5.25,158.5,1.82,1.92,-11.5,1.98,1.77,nFoUNZu1,0.854701,0.190476,0.549451,0.520833,0.045177,0.6,0.2,NaN,NaN,246.264,123.660196,0.502145,268.142,142.185527,0.530262,213.246,282.610,382.59,249.21,1.0,0.0,0.0,0.0,0.898753,0.037813,0.079196,2.43,0.486,0.349794,0.496072,0.5,0.003928,2.63,0.526,8.079848,0.274096,0.4,0.125904
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19574,2024-11-09,Macedônia Do Norte Prva Liga,15:00,Kozuv,MKK Kumanovo,1.32,3.19,160.5,1.83,1.83,-7.5,1.83,1.83,fHQs6sIQ,0.757576,0.313480,0.546448,0.546448,0.071055,0.8,0.0,NaN,NaN,194.188,99.573912,0.512771,NaN,NaN,NaN,222.348,0.000,162.40,0.00,0.0,0.0,0.0,0.0,0.586381,0.000000,0.000000,-3.02,-0.604,-0.529801,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
19575,2024-11-09,Portugal Proliga,14:00,Santo André,Benfica B,1.32,3.19,143.5,1.91,1.77,-8.5,1.96,1.75,2LQEiAuR,0.757576,0.313480,0.523560,0.564972,0.071055,0.2,0.4,NaN,NaN,118.692,22.416452,0.188862,139.796,21.793156,0.155893,114.996,162.308,109.56,151.83,0.0,0.0,0.0,0.0,0.586381,0.053802,0.080050,-0.20,-0.040,-8.000000,0.000000,0.0,0.000000,-0.39,-0.078,-28.076923,0.000000,0.0,0.000000
19576,2024-11-09,Portugal Lfb Feminina,12:30,Barcelos F,CAB Madeira F,1.37,2.90,134.5,1.83,1.83,-7.5,2.01,1.72,EHyGVOOp,0.729927,0.344828,0.546448,0.546448,0.074755,0.4,0.4,NaN,NaN,196.040,126.183891,0.643664,164.234,78.483566,0.477876,226.576,174.920,404.95,225.28,1.0,0.0,0.0,0.0,0.506732,0.000000,0.109952,-2.70,-0.540,-0.685185,0.000000,0.0,0.000000,-5.00,-1.000,-1.900000,0.000000,0.0,0.000000
19577,2024-11-09,República Tcheca Nbl,11:00,Brno,Srsni Pisek,2.08,1.69,164.5,1.85,1.85,-4.5,2.03,1.67,Cl

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,00:00,Eua Nba,Los Angeles Lakers,Philadelphia 76ers,223.5,1.84,0.9711,Over
1,00:00,Eua Nba,Sacramento Kings,Los Angeles Clippers,222.5,1.84,0.9849,Over
2,21:30,Eua Nba,Cleveland Cavaliers,Brooklyn Nets,223.5,1.80,0.5768,Over
3,14:00,França Lnb,Chalon/Saone,Le Mans,166.5,1.89,0.5249,Over
4,10:00,Rússia Liga Vtb United,Parma Perm,Nizhny Novgorod,151.5,1.82,0.9627,Over
5,20:00,Eua Ncaa,Norfolk State,James Madison,145.5,1.91,0.8429,Over
